In [2]:
!pip3 install keras

In [41]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving submit.py to submit.py
User uploaded file "submit.py" with length 1443 bytes


In [0]:
!unzip colab_upload.zip

In [3]:
!ls

A-test.csv	      data	   hist.zip	sample_data
colab_upload (1).zip  dataset.py   __pycache__	submit.py
colab_upload.zip      evaluate.py  quantize.py


In [1]:
from datetime import datetime, timedelta

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import backend as K
from keras.layers import Add, Dense, Dropout, Input
from keras.models import Model

import quantize
from dataset import get_hist_with_time
from evaluate import deploy_model, test_model
from submit import get_hour_density, submit_csv

Using TensorFlow backend.


In [0]:
is_test = True
img_size = 58 * 99
noise_size = 100

np.random.seed(2333)

In [0]:
def preprocess_data():
    if is_test:
        density, weekday, hours = get_hist_with_time(
            datetime(2017, 2, 6), datetime(2017, 3, 5))
    else:
        density, weekday, hours = get_hist_with_time(
            datetime(2017, 2, 6), datetime(2017, 3, 12))

    noise_samples = np.random.uniform(size=(density.shape[0], noise_size))
    hours = (hours.astype("float32") - 8) / 14.0
    weekday = (weekday.astype("float32") + 1) / 7.0
    #density = density.reshape((*density.shape, 1))
    train_img = np.array([
        img.flatten().astype("float32")/100.0 for img in density])

    
    X = [noise_samples, hours, weekday]
    y = train_img
    return X, y


In [0]:
def build_model():
    inputs_noise_img = Input(shape=(noise_size, ), name="noise_img")
    inputs_hour = Input(shape=(1, ), name="hour")
    inputs_weekday = Input(shape=(1, ), name="weekday")

    print(inputs_noise_img)

    noise_dense = Dense(64, activation="relu")(inputs_noise_img)
    hour_dense = Dense(64, activation="relu")(inputs_hour)
    weekday_dense = Dense(64, activation="relu")(inputs_weekday)
    hour_dense_128 = Dense(128, activation="relu")(inputs_hour)
    weekday_dense_128 = Dense(128, activation="relu")(inputs_weekday)
    x = Add()([noise_dense, hour_dense])
    x = Dense(64, activation="relu")(x)
    x = Add()([x, weekday_dense])
    x = Dense(64, activation="relu")(x)
    x = Add()([x, hour_dense])
    x = Dense(128, activation="relu")(x)
    x = Add()([x, weekday_dense_128])
    x = Dense(128, activation="relu")(x)
    x = Add()([x, hour_dense_128])
    x = Dense(256, activation="relu")(x)
    #x = keras.layers.concatenate([inputs_weekday, x])
    x = Dropout(0.2)(x)
    predictions = Dense(img_size)(x)
    model = Model(inputs=[inputs_noise_img, inputs_hour, inputs_weekday],
                  outputs=predictions)
    return model

In [19]:
if __name__ == "__main__":
    model = build_model()
    model.compile(optimizer="adam", loss='mean_squared_error')
    for i in range(5):
        X, y = preprocess_data()
        model.fit(x=X, y=y, epochs=5, batch_size=7
                  #,validation_split=0.05
                 )

    if is_test:
        errors = []
        date = datetime(2017, 3, 6)
        delta_day = timedelta(days=1)
        while date <= datetime(2017, 3, 12):
            for hour in range(9, 23):
                errors.append(test_model(model, date, hour))
            date += delta_day
        print("Avearage RMSE:{}".format(np.array(errors).mean()))
    else:
        frames = []
        date = datetime(2017, 3, 13)
        delta_day = timedelta(days=1)
        while date <= datetime(2017, 3, 26):
            for hour in range(9, 23):
                hist = deploy_model(model, date, hour)
                X_sub = get_hour_density(hist, date, hour)
                frames.append(X_sub)
            date += delta_day
        submission = pd.concat(frames).drop(
            ['latitude_range', 'longitude_range'], axis=1)
        submission.to_csv("./submission.csv", index=False)

Tensor("noise_img_8:0", shape=(?, 100), dtype=float32)
Epoch 1/5
392/392 [==============================] - 2s 5ms/step - loss: 8.8202e-04
Epoch 2/5
392/392 [==============================] - 1s 2ms/step - loss: 3.5025e-04
Epoch 3/5
392/392 [==============================] - 1s 2ms/step - loss: 3.1522e-04
Epoch 4/5
392/392 [==============================] - 1s 2ms/step - loss: 3.0198e-04
Epoch 5/5
392/392 [==============================] - 1s 2ms/step - loss: 2.8847e-04
Epoch 1/5
392/392 [==============================] - 1s 2ms/step - loss: 2.9982e-04
Epoch 2/5
392/392 [==============================] - 1s 2ms/step - loss: 2.8490e-04
Epoch 3/5
392/392 [==============================] - 1s 2ms/step - loss: 2.7866e-04
Epoch 4/5
392/392 [==============================] - 1s 2ms/step - loss: 2.7253e-04
Epoch 5/5
392/392 [==============================] - 1s 2ms/step - loss: 2.6881e-04
Epoch 1/5
392/392 [==============================] - 1s 2ms/step - loss: 2.7483e-04
Epoch 2/5
392/392 [==